In [110]:
import copy
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from Dataset import Dataset, ELM, device
from torch.utils.data import DataLoader
torch.set_default_dtype(torch.float64)

In [111]:
x_data = np.float64(np.linspace(-15, 15, 600))
# y_data = np.float64(np.sin(x_data) / x_data)
y_data = np.float64(np.sin(x_data) / x_data + np.random.normal(0, 0.1, 600))

In [112]:
dataset = Dataset(x_data, y_data)
trainset, testset = torch.utils.data.random_split(dataset, [0.9, 0.1], generator=torch.Generator().manual_seed(42))

trainloader = DataLoader(trainset, batch_size=256, shuffle=True)
testloader = DataLoader(testset, batch_size=256, shuffle=False)

activation = nn.Sigmoid()

In [113]:
# model = ELM(1, 300, 1).to(device)

# V = activation(model.fc1.bias.data.reshape(-1, 1) + model.fc1.weight.data @ trainset[:][0].reshape(1, -1))
# H = torch.concatenate([torch.ones(1, len(trainset)), V], dim=0).T
# T = trainset[:][1].reshape(1, -1).T
# W2 = torch.linalg.pinv(H) @ T.reshape(1, -1, 1)
# model.fc2.bias.data, model.fc2.weight.data = W2[0][0], W2[0][1:].T

In [114]:
def infGain(model, loader):
    model.eval()
    loss = 0
    for x, y in loader:
        x, y = x.reshape(-1, 1).to(device), y.reshape(-1, 1).to(device)
        preds = model(x).to(device)
        loss += torch.norm(preds - y) 
    return loss

def prune(model, loader):
    inf = []
    model.eval()
    with torch.no_grad():
        for i in range(model.fc1.out_features):
            fc1_weight_temp = copy.deepcopy(model.fc1.weight.data[i])
            fc1_bias_temp = copy.deepcopy(model.fc1.bias.data[i])
            fc2_weight_temp = copy.deepcopy(model.fc2.weight.data[:,i])
            model.fc1.weight.data[i] = 0
            model.fc1.bias.data[i] = 0
            model.fc2.weight.data[:,i] = 0
            inf.append(infGain(model, loader))
            model.fc1.weight.data[i] = fc1_weight_temp
            model.fc1.bias.data[i] = fc1_bias_temp
            model.fc2.weight.data[:,i] = fc2_weight_temp
        neuron_to_prune = np.argmin(torch.tensor(inf).cpu().detach().numpy())
        pruned_model = ELM(1, model.fc1.out_features-1, 1)
        pruned_model.fc1.weight.data = torch.cat([model.fc1.weight.data[:neuron_to_prune], model.fc1.weight.data[neuron_to_prune+1:]])
        pruned_model.fc1.bias.data = torch.cat([model.fc1.bias.data[:neuron_to_prune], model.fc1.bias.data[neuron_to_prune+1:]])
        pruned_model.fc2.weight.data = torch.cat([model.fc2.weight.data[:,:neuron_to_prune],model.fc2.weight.data[:,neuron_to_prune+1:]], dim=1)
    return pruned_model

# for _ in range(2):
#     model = prune(model, testloader).to(device)

# V = activation(model.fc1.bias.data.reshape(-1, 1) + model.fc1.weight.data @ trainset[:][0].reshape(1, -1))
# H = torch.concatenate([torch.ones(1, len(trainset)), V], dim=0).T
# T = trainset[:][1].reshape(1, -1).T
# W2 = torch.linalg.pinv(H) @ T.reshape(1, -1, 1)
# model.fc2.bias.data, model.fc2.weight.data = W2[0][0], W2[0][1:].T

In [115]:
# fig, ax = plt.subplots(1, 2, figsize=(15, 5))
max_train_error = []
min_train_error = []
max_test_error = []
min_test_error = []
num_neurons = 200
a = [5, 10, 50, 100, 500, 1000]
for j in a:
    print(j)
    train_error = []
    test_error = []
    for i in range(8):
        model = ELM(1, j, 1).to(device)
        V = activation(model.fc1.bias.data.reshape(-1, 1) + model.fc1.weight.data @ trainset[:][0].reshape(1, -1).to(device))
        H = torch.concatenate([torch.ones(1, len(trainset)).to(device), V.to(device)], dim=0).T
        T = trainset[:][1].reshape(1, -1).T.to(device)
        W2 = torch.linalg.pinv(H) @ T.reshape(1, -1, 1) # bez regularyzacji
        # P = torch.linalg.inv(H.T @ H + 1e-3*torch.eye(H.shape[1]))
        # W2 = P @ H.T @ T.reshape(1, -1, 1)
        model.fc2.bias.data, model.fc2.weight.data = W2[0][0], W2[0][1:].T
        for _ in range(max(int(j*0.02), 1)):
            model = prune(model, testloader).to(device)
        V = activation(model.fc1.bias.data.reshape(-1, 1) + model.fc1.weight.data @ trainset[:][0].reshape(1, -1).to(device))
        H = torch.concatenate([torch.ones(1, len(trainset)).to(device), V.to(device)], dim=0).T
        T = trainset[:][1].reshape(1, -1).T.to(device)
        W2 = torch.linalg.pinv(H) @ T.reshape(1, -1, 1)
        # P = torch.linalg.inv(H.T @ H + 1e-3*torch.eye(H.shape[1]))
        # W2 = P @ H.T @ T.reshape(1, -1, 1)
        model.fc2.bias.data, model.fc2.weight.data = W2[0][0], W2[0][1:].T
        train_error.append((model(trainset[:][0].clone().detach().reshape(-1, 1).to(device))-trainset[:][1].reshape(-1, 1).to(device)).reshape(-1).pow(2).mean())
        test_error.append((model(testset[:][0].clone().detach().reshape(-1, 1).to(device))-testset[:][1].reshape(-1, 1).to(device)).reshape(-1).pow(2).mean())
    max_train_error.append(max(train_error).item())
    min_train_error.append(min(train_error).item())
    max_test_error.append(max(test_error).item())
    min_test_error.append(min(test_error).item())
# ax[0].plot([5*i+5 for i in range(num_neurons)], max_train_error, 'r', label='Błąd maksymalny')
# ax[0].plot([5*i+5 for i in range(num_neurons)], min_train_error, 'b', label='Błąd minimalny')
# ax[0].set_ylabel('Błąd średniokwadratowy zbioru uczącego')
# ax[0].set_xlabel('Liczba neuronów')
# ax[0].set_yscale('log')
# ax[0].grid()
# ax[0].legend(loc='upper right')
# ax[1].plot([5*i+5 for i in range(num_neurons)], max_test_error, 'r', label='Błąd maksymalny')
# ax[1].plot([5*i+5 for i in range(num_neurons)], min_test_error, 'b', label='Błąd minimalny')
# ax[1].set_ylabel('Błąd średniokwadratowy zbioru weryfikującego')
# ax[1].set_xlabel('Liczba neuronów')
# ax[1].set_yscale('log')
# ax[1].grid()
# ax[1].legend(loc='upper right')
# plt.show()
from decimal import Decimal
for i in min_test_error:
    print(f'{Decimal(i):.2e}', end=' & ')

5


10
50
100
500
1000
1.72e-2 & 1.51e-2 & 9.79e-3 & 9.68e-3 & 9.68e-3 & 9.69e-3 & 

In [116]:
print(f'Train error: {(model(trainset[:][0].clone().detach().reshape(-1, 1))-trainset[:][1].reshape(-1, 1)).reshape(-1).pow(2).mean()}')
print(f'Test error: {(model(testset[:][0].clone().detach().reshape(-1, 1))-testset[:][1].reshape(-1, 1)).reshape(-1).pow(2).mean()}')
plt.plot(x_data, y_data)
plt.plot(x_data, model(torch.tensor(x_data).reshape(-1, 1)).detach().numpy(), 'r--')
plt.legend(['Dane', 'Model'], loc='upper right')
plt.grid()
plt.show()

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [ ]:
plt.plot([np.min(y_data), np.max(y_data)], [np.min(y_data), np.max(y_data)])
plt.plot(y_data, model(torch.tensor(x_data).reshape(-1, 1)).detach().numpy(), '.', markersize=1)
plt.grid()
plt.show()